In [4]:
!pwd

### Context:
In this assignment I tried to address the following question:
Can a quantitative model depict, and de-cypher gentrification and its drivers?

While there was no sufficient available data for my desired area of investigation (Jerusalem)* , I reckoned that there will be rich dataset for this investigation for other cities, based on which some conclusions (but not all) could be generalized; especially so, if I were to tackle the question from different angels - looking at many correlated varuables, and perhaps even for two different cities.

The lack of the perfect dataest for Jerusalem is perhaps not so surprising. As it was put by AUTHOR in Requiem for Large Scale Models: 'CITATION about lack of granular data'. While we are seeing an increase in relevant data all over the globe, yet this pitfall is still very real.

### Exploratory Analysis:

In [1]:
import pandas as pd

HOUSING = './Data/2014_Housing_Market_Analysis_Data_by_Zip_Code.csv'
GENTRIFICATION = './UrbanModelling-Segragation/Austin/Data/Displacement_and_Gentrification_Recommendation_Inventory.csv'
SHOOTING = './UrbanModelling-Segragation/Austin/Data/gfa7-xe7p.csv'
housingDF = pd.read_csv(HOUSING)
housingDF.head()

,Zip Code,Population below poverty level,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",...,Rental units affordable to average teacher,Owner units affordable to average teacher,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related
0,78726.0,9,66096,16,20,3,4,2,0.0,1,...,70,0.0,99,17.0,375,20,55,2,786,40
1,NaN,19,52431,16,35,8,7,8,24.0,10,...,74,22.0,98,58.0,65,31,74,29,668,38
2,78724.0,38,35711,29,63,8,8,28,85.0,16,...,78,84.0,100,96.0,208,37,46,21,786,40
3,78617.0,18,43957,12,67,10,15,23,NaN,11,...,63,NaN,99,NaN,101,74,21,16,865,42
4,78701.0,20,68152,16,14,10,9,0,7.0,7,...,29,7.0,90,30.0,12,115,59,97,433,23


In [6]:
# extract the zip codes for external integration with maps:
zipC = housingDF['Zip Code']

In [12]:
# split the data into train and test:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    housingDF.loc[:,housingDF.columns !='Population below poverty level'],
    housingDF['Population below poverty level'], test_size=0.33, random_state=42)

X_train

,Zip Code,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",Rent-restricted units,...,Rental units affordable to average teacher,Owner units affordable to average teacher,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related
32,78754.0,53274,36,32,8,9,7,70.0,4,9,...,74,59.0,100,100.0,286,33,33,19,708,41
9,78717.0,93305,25,14,4,5,9,7.0,1,0,...,70,6.0,98,67.0,638,2,33,3,708,47
19,78735.0,74571,9,20,7,4,10,8.0,4,5,...,62,5.0,97,23.0,96,4,85,9,747,38
0,78726.0,66096,16,20,3,4,2,0.0,1,0,...,70,0.0,99,17.0,375,20,55,2,786,40
31,78753.0,39593,20,60,10,9,16,78.0,14,17,...,89,75.0,100,98.0,128,26,40,59,708,44
5,78702.0,34734,18,56,14,11,10,21.0,41,39,...,80,15.0,99,67.0,3,73,207,96,590,39
11,78722.0,44917,18,19,11,8,3,7.0,11,12,...,65,6.0,97,37.0,8,40,127,98,590,33
1,NaN,52431,16,35,8,7,8,24.0,10,10,...,74,22.0,98,58.0,65,31,74,29,668,38
27,78749.0,80956,13,20,7,4,7,5.0,1,1,...,57,3.0,97,70.0,171,25,61,37,708,40
21,78741.0,30183,16,62,9,9,10,76.0,15,21,...,86,76.0,100,93.0,45,28,58,80,668,43


In [ ]:
from sklearn.linear_model import LinearRegression as lm
linearModel = lm()
linearModel.fit(X_train,y_train)

In [ ]:
gentrificationDF = pd.read_csv(GENTRIFICATION)
gentrificationDF.head()

In [ ]:
shootingDF = pd.read_csv(SHOOTING)
shootingDF.head()

In [ ]:
lonLat = officersDF['location_1']
lonLat

### Modeling - Hypothesis and Analysis

### Modeling - Assumptions and Limitations

### HCs / LOs:
 * DatavizandCom (CompUrbanism) - in regards to this project, data visualization and communication has been a journey.
* model assunmptions something about the large scale models and biased data etc...